In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import h5py
import os

sns.set_theme('poster')

def GetTemplateFiles(config):
    template_path = '/storage/home/adz6/group/project/datasets/data/dense_template_grid'
    
    radii, angles = np.meshgrid(config['r'], config['pa'])
    
    template_list = []
    for i, param_grp in enumerate(zip(radii.flatten(), angles.flatten())):
        for j, template_file in enumerate(os.listdir(template_path)):
            #scores_name = score_path.split('/')[-1]

            angle = template_file.split('grid_')[-1].split('_')[0]
            rad = template_file.split('cm.h5')[0].split('_')[-1]
            
            if (param_grp[0] == int(rad)) and (param_grp[1] == float(angle)):
                #print(template_file)
                template_list.append(os.path.join(template_path, template_file))

    return template_list

config = {
    'r': [1,],
    'pa': [86.0,87.0, 88.0],
}


def ModifySignalLength(signal, sig_length, length):
    
    signal_array = np.copy(signal)[:, 0:length] # np.zeros((signal.shape[0], length), dtype=np.complex64)
    
    if sig_length <= length:
    
        signal_array[:, sig_length:] = 0
        
    return signal_array


def ModifySignalStartTime(signal, diff, length):
    
    signal_array = np.zeros((signal.shape[0], length), dtype=np.complex64)
    
    if diff >= 0:
        signal_array[:, diff:] = signal[:, 0:length-diff]
    elif diff < 0:
        signal_array[:, :] = signal[:, -diff:length-diff]
        
    return signal_array


def ModifySignal(signal, sig_length, diff, length):
    
    # if the signal is long enough such that it will fill the template
    # then we only need to shift start time
    if (sig_length >= (length - diff)):
        
        mod_signal = ModifySignalStartTime(signal, diff, length)
        return mod_signal
    # if the signal is shorter than the negative start time offset 
    # combined with template length it will end before the template
    # then we shift the signal and change the length
    elif (sig_length < (length - diff)):
        
        mod_signal = ModifySignalStartTime(signal, diff, length)
        # after the shift the length of the signal in the template
        # window will be (sig_length + diff) since diff is negative
        mod_signal = ModifySignalLength(mod_signal, sig_length + diff, length)
        
        return mod_signal
    # if the start shift is positive and the signal is shorter than the remaining template window
    # we shift then shorten the signal
    #elif (sig_length < (length - diff)) and (diff >= 0):
        
    #    mod_signal = ModifySignalStartTime(signal, diff, length)
    #    mod_signal = ModifySignalLength(mod_signal, sig_length + diff, length)
        
    #    return mod_signal
        

In [ ]:
template_file_paths = GetTemplateFiles(config)
templates = h5py.File(template_file_paths[0], 'r')

In [ ]:
rng = np.random.default_rng()
rand_int = rng.integers(0, templates['data'].shape[0], 1)

template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

# check signal modifications

## posititve start time but long signal

In [ ]:
sig_length = 2 * 8192
length = 8192
start_time_diff = 0

mod_signal = ModifySignal(signal, sig_length, start_time_diff, length)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(mod_signal.real[0, :])
ax.set_xlabel('Sample')
ax.set_ylabel('V')

In [ ]:
sig_length = 2 * 8192
length = 8192
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
for start_time_diff in [0, 1024, 4096]:
    #start_time_diff = 4096

    mod_signal = ModifySignal(signal, sig_length, start_time_diff, length)

    ax.plot(abs(np.fft.fftshift(np.fft.fft(mod_signal[0, :]))) / length, label = fr'$\Delta N$ = {start_time_diff}')
ax.set_xlabel('Arb Freq.')
ax.set_ylabel('V')

ax.legend(loc=1)

## positive start time but short signal

In [ ]:
sig_length = 1 * 4096
length = 8192
start_time_diff = 1024

mod_signal = ModifySignal(signal, sig_length, start_time_diff, length)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(mod_signal.real[0, :])

ax.set_xlabel('Sample')
ax.set_ylabel('V')

## negative start time but long signal

In [ ]:
sig_length = 8192 * 2
length = 8192
start_time_diff = -1024

mod_signal = ModifySignal(signal, sig_length, start_time_diff, length)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(mod_signal.real[0, :])

## negative start time but short signal

In [ ]:
sig_length = 4096 * 1
length = 8192
start_time_diff = -1024

mod_signal = ModifySignal(signal, sig_length, start_time_diff, length)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(mod_signal.real[0, :])

# Compute curves for modified signals

## fix the start time offset, but have many signal lengths

In [ ]:
start_time_diff = 100
# only signal_lengths greater than the negative start time diff. make sense to look at
length = 8192
var = 1.38e-23 * 10 * 50 * 200e6  / length
signal_lengths = np.int32(np.linspace(100, 2 * 8192, 101))

rng = np.random.default_rng()
rand_int = rng.integers(0, templates['data'].shape[0], 1)

template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

template = np.fft.fft(template[:, 0:length], axis = -1) / (length)
norm = 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))
template = norm * template

score_array = np.zeros(signal_lengths.size)

for i, sig_length in enumerate(signal_lengths):
    
    mod_signal = ModifySignal(signal, sig_length, start_time_diff, length)
    
    mod_signal = np.fft.fft(mod_signal) / length
    
    score_array[i] = abs(mod_signal.conjugate() * template).sum()
    
    
    
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(signal_lengths, score_array)

## multiple start times with many signal lengths

In [ ]:
start_time_diff_list = [256, 512, 1024, 2048, 4096]
# only signal_lengths greater than the negative start time diff. make sense to look at
length = 8192
var = 1.38e-23 * 10 * 50 * 200e6  / length
signal_lengths = np.int32(np.linspace(1, 2 * 8192, 201))

rng = np.random.default_rng()
rand_int = rng.integers(0, templates['data'].shape[0], 1)

template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

template = np.fft.fft(template[:, 0:length], axis = -1) / (length)
norm = 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))
template = norm * template

colors = sns.color_palette('deep', len(start_time_diff_list) + 1)
ideal_score_array = np.zeros(signal_lengths.size)

# compute the ideal scores
for i, sig_length in enumerate(signal_lengths):

    full_signal = ModifySignalLength(signal, sig_length, sig_length)

    norm = abs( 1 / np.sqrt(length * var * np.vdot(full_signal.flatten(), full_signal.flatten())))
    ideal_score_array[i] = abs(norm * full_signal.conjugate() * full_signal).sum()
    
    

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for n, start_time_diff in enumerate(start_time_diff_list):
    
    score_array = np.zeros(signal_lengths.size)
    
    for i, sig_length in enumerate(signal_lengths):

        signal_in_window = ModifySignal(signal, sig_length, start_time_diff, length)

        signal_in_window = np.fft.fft(signal_in_window, axis=-1) / length
        
        score_array[i] = abs(signal_in_window.conjugate() * template).sum()

    ax.plot(signal_lengths, score_array, color = colors[n], label=fr'Start $\Delta N$ = {start_time_diff}')
    #ax.plot(signal_lengths, ideal_score_array, color = colors[i])
    
ax.plot(signal_lengths, ideal_score_array, color = colors[len(start_time_diff_list)], label='Ideal')

ax.legend(loc=(1.01, 0.3))
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Score')


In [ ]:
start_time_diff_list = [256, 512, 1024, 2048, 4096]
# only signal_lengths greater than the negative start time diff. make sense to look at
length = 8192
var = 1.38e-23 * 10 * 50 * 200e6  / length
signal_lengths = np.int32(np.linspace(1, 2 * 8192, 201))

rng = np.random.default_rng()
rand_int = rng.integers(0, templates['data'].shape[0], 1)

template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

template = np.fft.fft(template[:, 0:length], axis = -1) / (length)
norm = 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))
template = norm * template

colors = sns.color_palette('deep', len(start_time_diff_list) + 1)
ideal_score_array = np.zeros(signal_lengths.size)

# compute the ideal scores
for i, sig_length in enumerate(signal_lengths):

    full_signal = ModifySignalLength(signal, sig_length, sig_length)

    norm = abs( 1 / np.sqrt(length * var * np.vdot(full_signal.flatten(), full_signal.flatten())))
    ideal_score_array[i] = abs(norm * full_signal.conjugate() * full_signal).sum()
    
    

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for n, start_time_diff in enumerate(start_time_diff_list):
    
    score_array = np.zeros(signal_lengths.size)
    
    for i, sig_length in enumerate(signal_lengths):

        signal_in_window = ModifySignal(signal, sig_length, start_time_diff, length)

        signal_in_window = np.fft.fft(signal_in_window, axis=-1) / length
        
        score_array[i] = abs(signal_in_window.conjugate() * template).sum()

    ax.plot(signal_lengths, score_array / ideal_score_array, color = colors[n], label=fr'Start $\Delta N$ = {start_time_diff}')
    #ax.plot(signal_lengths, ideal_score_array, color = colors[i])
    
#ax.plot(signal_lengths, ideal_score_array, color = colors[len(start_time_diff_list)])

ax.legend(loc=(1.01, 0.3))
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Score')




## plot a grid of start time offsets and signal lengths

In [ ]:
ngrid = 41

start_time_diff_list = np.int32(np.linspace(0, 4096, ngrid))
# only signal_lengths greater than the negative start time diff. make sense to look at
length = 8192
var = 1.38e-23 * 10 * 50 * 200e6  / length
signal_lengths = np.int32(np.linspace(1, 2 * 8192, ngrid))

rng = np.random.default_rng()
rand_int = rng.integers(0, templates['data'].shape[0], 1)

template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

template = np.fft.fft(template[:, 0:length], axis = -1) / (length)
norm = 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))
template = norm * template

colors = sns.color_palette('deep', len(start_time_diff_list) + 1)
ideal_score_array = np.zeros(signal_lengths.size)

score_grid = np.zeros((ngrid, ngrid))

# compute the ideal scores
for i, sig_length in enumerate(signal_lengths):

    full_signal = ModifySignalLength(signal, sig_length, sig_length)

    norm = abs( 1 / np.sqrt(length * var * np.vdot(full_signal.flatten(), full_signal.flatten())))
    ideal_score_array[i] = abs(norm * full_signal.conjugate() * full_signal).sum()
    
    
sns.set_theme(context='poster', style='ticks')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

for n, start_time_diff in enumerate(start_time_diff_list):
    
    score_array = np.zeros(signal_lengths.size)
    
    for i, sig_length in enumerate(signal_lengths):

        signal_in_window = ModifySignal(signal, sig_length, start_time_diff, length)

        signal_in_window = np.fft.fft(signal_in_window, axis=-1) / length
        
        #score_array[i] = abs(signal_in_window.conjugate() * template).sum()
        score_grid[n, i] = abs(signal_in_window.conjugate() * template).sum()

    #ax.plot(signal_lengths, score_array / ideal_score_array, color = colors[n])
    #ax.plot(signal_lengths, ideal_score_array, color = colors[i])
    
#ax.plot(signal_lengths, ideal_score_array, color = colors[len(start_time_diff_list)])

cmap = sns.color_palette('mako_r', as_cmap=True)

img = ax.imshow(
    score_grid,
    aspect='auto',
    extent=(signal_lengths[0], signal_lengths[-1], start_time_diff_list[-1], start_time_diff_list[0]),
    cmap=cmap
)
cbar = fig.colorbar(img, label='Score')
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Start Time Offset (Samples)')


fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(
    score_grid / ideal_score_array.reshape((1, ngrid)),
    aspect='auto',
    extent=(signal_lengths[0], signal_lengths[-1], start_time_diff_list[-1], start_time_diff_list[0]),
    cmap=cmap
)
cbar = fig.colorbar(img, label='Match')
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Start Time Offset (Samples)')


